# Import and load

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
import seaborn as sns
import missingno as msno
import plotly.express as px

from tqdm import tqdm

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error

from keras.callbacks import EarlyStopping

from keras.layers import Dense, LSTM, Dropout
from keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler


In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# Feature engineering + Lag

In [3]:
def create_lag(df, num_lags):
    df['year'] = pd.to_datetime(df['Date']).dt.year
    df['month'] = pd.to_datetime(df['Date']).dt.month
    df['day'] = pd.to_datetime(df['Date']).dt.day
    df['Date'] = pd.to_datetime(df['Date'])    
    df.set_index('Date', inplace=True)
    df.sort_values(by=['ID', 'Date'], inplace=True)
    for i in range(1, num_lags+1):
        df[f'lag{i}'] = df.groupby('ID')['GT_NO2'].shift(i)

    return df

def full_df(df):
    df = df.copy().reset_index()
    df = df.drop(['ID', 'ID_Zindi', 'Date', 'NO2_trop'],axis=1)
    return df

In [4]:
trained = create_lag(train, 15)
trained = full_df(trained)

C:\Users\leeno\AppData\Local\Temp\ipykernel_20696\3738387936.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['year'] = pd.to_datetime(df['Date']).dt.year
C:\Users\leeno\AppData\Local\Temp\ipykernel_20696\3738387936.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['month'] = pd.to_datetime(df['Date']).dt.month
C:\Users\leeno\AppData\Local\Temp\ipykernel_20696\3738387936.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['day'] = pd.to_datetime(df['Date']).dt.day
C:\Users\leeno\AppData\Local\Temp\ipykernel_20696\3738387936.py:5: UserWarning: Could not infer f

# Model training + validation

* Min RMSE

In [5]:
rts = TimeSeriesSplit(max_train_size=5000)

sub_set = trained.dropna()

x = sub_set.drop('GT_NO2', axis=1)
y = sub_set['GT_NO2']

In [6]:
seq_length = 15

def create_sequences(data, labels, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x_seq = data[i:i+seq_length]
        y_seq = labels.iloc[i+seq_length]
        xs.append(x_seq)
        ys.append(y_seq)
    return np.array(xs), np.array(ys)

scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(x)
X, y = create_sequences(scaled_features, y, seq_length)

In [7]:
def my_model(input_shape):
    model = Sequential()
    model.add(LSTM(128, activation='relu', input_shape=input_shape, return_sequences=True))
    model.add(Dropout(0.2))  # Dropout layer added
    model.add(LSTM(64, activation='relu', return_sequences=False))
    model.add(Dropout(0.2))  # Dropout layer added
    model.add(Dense(32, activation='relu'))
    model.add(Dense(16, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')
    return model

input_shape = (seq_length, X.shape[2])
model = my_model(input_shape)
model.summary()



Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 15, 128)           79872     
                                                                 
 dropout (Dropout)           (None, 15, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dense_1 (Dense)             (None, 16)                528       
                                                                 
 dense_2 (Dense)             (None, 1)                

In [8]:
# Initialize TimeSeriesSplit with max_train_size
tscv = TimeSeriesSplit(max_train_size=5000)

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Training and evaluation loop
for fold, (train_index, test_index) in enumerate(tscv.split(X), 1):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    
    model = my_model(input_shape)
    
    # Train the model with early stopping and validation data
    history = model.fit(
        X_train, y_train,
        epochs=200,
        batch_size=32,
        verbose=1,
        validation_split=0.1,  # Use 10% of training data for validation
        callbacks=[early_stopping]
    )
    
    # Print training loss and validation loss from the last epoch
    train_loss = history.history['loss'][-1]
    val_loss = history.history['val_loss'][-1]
    print(f'Fold {fold} - Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}')
    
    # Evaluate the model on the test set
    y_pred = model.predict(X_test)
    test_loss = mean_squared_error(y_test, y_pred)
    print(f'Fold {fold} - Test Loss (MSE): {test_loss:.4f}')

Epoch 1/200

141/141 [==============================] - 6s 24ms/step - loss: 207.0643 - val_loss: 102.5765
Epoch 2/200
141/141 [==============================] - 3s 21ms/step - loss: 134.3541 - val_loss: 99.1838
Epoch 3/200
141/141 [==============================] - 3s 21ms/step - loss: 126.7472 - val_loss: 106.6983
Epoch 4/200
141/141 [==============================] - 3s 20ms/step - loss: 124.2510 - val_loss: 98.4497
Epoch 5/200
141/141 [==============================] - 3s 19ms/step - loss: 125.4161 - val_loss: 103.9925
Epoch 6/200
141/141 [==============================] - 3s 19ms/step - loss: 120.9428 - val_loss: 96.4367
Epoch 7/200
141/141 [==============================] - 3s 19ms/step - loss: 122.1383 - val_loss: 95.5044
Epoch 8/200
141/141 [==============================] - 3s 19ms/step - loss: 118.5371 - val_loss: 98.7461
Epoch 9/200
141/141 [==============================] - 3s 20ms/step - loss: 117.1025 - val_loss: 96.8504
Epoch 10/200
141/141 [=============================

In [16]:
len(history.history['loss'])


21